<h1>Nhóm 3</h1>

# Tiền xử lý dữ liệu

## Xem dataset

### Import thư viện

In [1]:
import pandas as pd
import glob
# from fuzzywuzzy import process
import unidecode

### Lấy tất cả file xlsx

In [ ]:
files = glob.glob(r'*.xlsx')
for file in files:
    print(file)

### Thông tin các files trong dataset

In [ ]:
pd.set_option('display.max_columns', None)
for file in files:
    df = pd.read_excel(file)
    print(f"File: {file}\n")
    print(df.info())
    print("\n" + "-"*20 + "\n")
    print(df.describe())
    print("\n" + "="*40 + "\n")

File: 01.sinhvien.xlsx

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8296 entries, 0 to 8295
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mssv          8295 non-null   object 
 1   gioitinh      8295 non-null   float64
 2   lopsh         8295 non-null   object 
 3   khoahoc       8295 non-null   float64
 4   chuyennganh2  8295 non-null   object 
 5   hedt_ CLC     8296 non-null   bool   
 6   hedt_ CNTN    8296 non-null   bool   
 7   hedt_ CQUI    8296 non-null   bool   
 8   hedt_ CTTT    8296 non-null   bool   
 9   hedt_ KSTN    8296 non-null   bool   
 10  khoa_ CNPM    8296 non-null   bool   
 11  khoa_ HTTT    8296 non-null   bool   
 12  khoa_ KHMT    8296 non-null   bool   
 13  khoa_ KTMT    8296 non-null   bool   
 14  khoa_ KTTT    8296 non-null   bool   
 15  khoa_ MMT&TT  8296 non-null   bool   
 16  khuvuc        8296 non-null   int64  
dtypes: bool(11), float64(2), int64(1), object(3)
me

### Xử lý các cột linh tinh trong tất cả file

In [ ]:
for file in files:
    df = pd.read_excel(file)

    # Xóa khoảng trẳng header
    df.columns = df.columns.str.strip()

    # Xóa mấy cột linh tinh
    cols_to_drop = [col for col in df.columns if (col.startswith('_') and col[1:].isdigit()) or col == "Column1"]
    df.drop(columns=cols_to_drop, inplace=True)

    # Xóa mấy dòng bị trùng ( em kh biết nên xóa kh nx )
    df.drop_duplicates(inplace=True)

    # Xóa mấy ký tự đặc biệt '' "" để dữ liệu nhất quán và các giá trị String nma NULL
    df = df.apply(lambda x: x.replace("'", "").replace('"', '') if isinstance(x, str) else x)
    df.replace(['NULL', 'null', 'Null', 'None', ' NULL'], pd.NA, inplace=True)

### Xử lý file thí sinh


<h3>Hàm để clean cái khu vực ngoại lai và match những khu vực có tỉ lệ giống 50% so với map</h3>

In [ ]:
def cleaning(name):
    if pd.isna(name) or name.strip() == "":
        return ""
    name = name.strip()
    name = unidecode.unidecode(name).title()
    name = name.replace("'", "").replace('"', '').strip()
    return name

def get_closest_match(name):
    cleaned_name = cleaning(name)
    if not cleaned_name:
        return None
    match, score = process.extractOne(cleaned_name, mapp_flat)
    return match if score >= 50 else None

<h3> Mapping khu vực đối với khu vực ngoài thì giá trị -1 </h3>

In [ ]:
vung_kinh_te ={
    0: ['Hòa Bình', 'Sơn la', 'Điện Biên', 'Lai Châu', 'Lào Cai', 'Yên Bái','Phú Thọ', 'Hà Giang', 'Tuyên Quang', 'Cao Bằng', 'Bắc Kạn', 'Thái Nguyên Lạng Sơn', 'Bắc Giang', 'Quảng Ninh'],
    1: ['Hà Nội', 'Bắc Ninh', 'Hải Phòng', 'Hải Dương', 'Hải Nam', 'Nam Định', 'Hưng Yên', 'Vĩnh Phúc', 'Thái Bình', 'Ninh Bình'],
    2: ['Thanh Hóa', 'Nghệ An', 'Hà Tĩnh', 'Quảng Bình', 'Quảng Trị', 'Thừa Thiên Huế'],
    3: ['Đà Nẵng', 'Quảng Nam', 'Quảng Ngãi', 'Bình Định', 'Phú Yên', 'Khánh Hòa', 'Bình Thuận', 'Ninh Thuận'],
    4: ['Kon Tum', 'Gia Lai', 'Đắk Lắk', 'Đắk Nông', 'Lâm Đồng', 'Daklak'],
    5: ['Thành phố Hồ Chí Minh', 'Bà Rịa – Vũng Tàu', 'Bình Dương', 'Bình Phước', 'Đồng Nai', 'Tây Ninh'],
    6: ['Cần Thơ', 'Long An', 'Tiền Giang', 'Bến Tre', 'Vĩnh Long', 'Trà Vinh', 'Hậu Giang', 'Sóc Trăng', 'Đồng Tháp', 'An Giang', 'Kiên Giang', 'Bạc Liêu, Cà Mau.']
}

reverse_mapping = {province: region for region, provinces in vung_kinh_te.items() for province in provinces}
mapp_flat = [t for tinh in vung_kinh_te.values() for t in tinh]


In [ ]:
for file in files:
    df = pd.read_excel(file)

    # One hot encoding

    if 'hedt' in df.columns:
        df = pd.get_dummies(df, columns=['hedt'], prefix='hedt')
    if 'khoa' in df.columns:
        df = pd.get_dummies(df, columns=['khoa'], prefix='khoa')

    if 'noisinh' in df.columns:
        df['noisinh'] = df['noisinh'].apply(cleaning)
        df['khuvuc'] = df['noisinh'].apply(get_closest_match)
        df['khuvuc'] = df['khuvuc'].map(reverse_mapping)
        df['khuvuc'] = df['khuvuc'].fillna(-1).astype('Int64')
        unexpected_values = df[df['khuvuc'] == -1]['noisinh'].unique()
        if unexpected_values.size > 0:
            print(f"Test Error: {unexpected_values}")
    df.to_excel(file, index=False)


In [ ]:
for file in files:
    df = pd.read_excel(file)

    # One hot encoding
    if 'dien_tt' in df.columns and 'diem_tt' in df.columns:
        df['xeploai_tt'] = None

        for index, row in df.iterrows():
            if row['dien_tt'] == 'ĐGNL':
                dgnl_score = row['diem_tt']
                if dgnl_score < 600:
                    df.at[index, 'xeploai_tt'] = 0
                elif 600 <= dgnl_score < 750:
                    df.at[index, 'xeploai_tt'] = 1
                elif 750 <= dgnl_score < 900:
                    df.at[index, 'xeploai_tt'] = 2
                elif 900 <= dgnl_score < 1000:
                    df.at[index, 'xeploai_tt'] = 3
                elif dgnl_score >= 1000:
                    df.at[index, 'xeploai_tt'] = 4
            else:
                thpt_score = row['diem_tt']
                if thpt_score < 20:
                    df.at[index, 'xeploai_tt'] = 0
                elif 20 <= thpt_score < 22:
                    df.at[index, 'xeploai_tt'] = 1
                elif 22 <= thpt_score < 24:
                    df.at[index, 'xeploai_tt'] = 2
                elif 24 <= thpt_score < 26:
                    df.at[index, 'xeploai_tt'] = 3
                elif thpt_score >= 26:
                    df.at[index, 'xeploai_tt'] = 4

    if 'drl' in df.columns:
        df['drltl'] = None

        for index, row in df.iterrows():
            drl_score = row['drl']
            if drl_score < 35:
                df.at[index, 'drltl'] = 0
            elif 35 <= drl_score < 50:
                df.at[index, 'drltl'] = 1
            elif 50 <= drl_score < 65:
                df.at[index, 'drltl'] = 2
            elif 65 <= drl_score < 80:
                df.at[index, 'drltl'] = 3
            elif 80 <= drl_score < 90:
                df.at[index, 'drltl'] = 4
            elif drl_score >= 90:
                df.at[index, 'drltl'] = 5

    if 'hedt' in df.columns:
        df = pd.get_dummies(df, columns=['hedt'], prefix='hedt')
    if 'khoa' in df.columns:
        df = pd.get_dummies(df, columns=['khoa'], prefix='khoa')

    if 'noisinh' in df.columns:
        df['noisinh'] = df['noisinh'].apply(cleaning)
        df['khuvuc'] = df['noisinh'].apply(get_closest_match)
        df['khuvuc'] = df['khuvuc'].map(reverse_mapping)
        df['khuvuc'] = df['khuvuc'].fillna(-1).astype('Int64')
        unexpected_values = df[df['khuvuc'] == -1]['noisinh'].unique()
        if unexpected_values.size > 0:
            print(f"Test Error: {unexpected_values}")
    df.to_excel(file, index=False)


In [ ]:
for file in files:
 if file in '01.sinhvien.xlsx':
    df = pd.read_excel(file)
    tmp = ['id','namsinh','noisinh','diachi_tinhtp','tinhtrang']
    df.drop(columns=tmp, inplace=True, errors='ignore')
    df.info()
    df.to_excel(file, index=False)
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8296 entries, 0 to 8295
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mssv          8295 non-null   object 
 1   gioitinh      8295 non-null   float64
 2   lopsh         8295 non-null   object 
 3   khoahoc       8295 non-null   float64
 4   chuyennganh2  8295 non-null   object 
 5   hedt_ CLC     8296 non-null   bool   
 6   hedt_ CNTN    8296 non-null   bool   
 7   hedt_ CQUI    8296 non-null   bool   
 8   hedt_ CTTT    8296 non-null   bool   
 9   hedt_ KSTN    8296 non-null   bool   
 10  khoa_ CNPM    8296 non-null   bool   
 11  khoa_ HTTT    8296 non-null   bool   
 12  khoa_ KHMT    8296 non-null   bool   
 13  khoa_ KTMT    8296 non-null   bool   
 14  khoa_ KTTT    8296 non-null   bool   
 15  khoa_ MMT&TT  8296 non-null   bool   
 16  khuvuc        8296 non-null   int64  
dtypes: bool(11), float64(2), int64(1), object(3)
memory usage: 478.1+ KB


### Xử lý file xếp loại anh văn

In [ ]:
for file in files:
   if file in '04.xeploaiav.xlsx':
      df = pd.read_excel(file)
      tmp = ['id','listening','reading','mamh','ghichu']
      df.drop(columns=tmp, inplace=True, errors='ignore')
      df.info()
      df.to_excel(file, index=False)
      break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6344 entries, 0 to 6343
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    6343 non-null   object 
 1   total   6343 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.3+ KB


In [ ]:
for file in files:
    if file == '04.xeploaiav.xlsx':
        df = pd.read_excel(file)
        df['xl_av'] = None

        for index, row in df.iterrows():
            total = row['total']
            if total < 40:
                df.at[index, 'xl_av'] = 0
            elif 40 <= total < 60:
                df.at[index, 'xl_av'] = 1
            elif 60 <= total < 70:
                df.at[index, 'xl_av'] = 2
            elif 70 <= total < 80:
                df.at[index, 'xl_av'] = 3
            elif 80 <= total < 90:
                df.at[index, 'xl_av'] = 4
            elif total >= 90:
                df.at[index, 'xl_av'] = 5
        df.drop('total', inplace=True,axis=1)

        df.info()

        df.to_excel(file, index=False)
        break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6344 entries, 0 to 6343
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    6343 non-null   object 
 1   xl_av   6343 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.3+ KB


### Xử lý file trúng tuyển

<h3>Tạo cột xeploai_tt dựa trên dien_tt và diem_tt</h3>

In [ ]:
for file in files:
    df = pd.read_excel(file)

    # One hot encoding
    if 'dien_tt' in df.columns and 'diem_tt' in df.columns:
        df['xeploai_tt'] = None

        for index, row in df.iterrows():
            if row['dien_tt'] == 'ĐGNL':
                dgnl_score = row['diem_tt']
                if dgnl_score < 600:
                    df.at[index, 'xeploai_tt'] = 0
                elif 600 <= dgnl_score < 750:
                    df.at[index, 'xeploai_tt'] = 1
                elif 750 <= dgnl_score < 900:
                    df.at[index, 'xeploai_tt'] = 2
                elif 900 <= dgnl_score < 1000:
                    df.at[index, 'xeploai_tt'] = 3
                elif dgnl_score >= 1000:
                    df.at[index, 'xeploai_tt'] = 4
            else:
                thpt_score = row['diem_tt']
                if thpt_score < 20:
                    df.at[index, 'xeploai_tt'] = 0
                elif 20 <= thpt_score < 22:
                    df.at[index, 'xeploai_tt'] = 1
                elif 22 <= thpt_score < 24:
                    df.at[index, 'xeploai_tt'] = 2
                elif 24 <= thpt_score < 26:
                    df.at[index, 'xeploai_tt'] = 3
                elif thpt_score >= 26:
                    df.at[index, 'xeploai_tt'] = 4


In [ ]:
for file in files:
   if file in '05.ThiSinh.xlsx':
      df = pd.read_excel(file)
      tmp = ['lop12_matinh','lop12_matruong','TEN_TRUONG','dien_tt','diem_tt']
      df.drop(columns=tmp, inplace=True, errors='ignore')
      df.info()
      df.to_excel(file, index=False)
      break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8234 entries, 0 to 8233
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   mssv        8234 non-null   object 
 1   xeploai_tt  8104 non-null   float64
dtypes: float64(1), object(1)
memory usage: 128.8+ KB


### Xử lý file điểm rèn luyện

In [ ]:
for file in files:
    df = pd.read_excel(file)

    # One hot encoding

    if 'drl' in df.columns:
        df['drltl'] = None

        for index, row in df.iterrows():
            drl_score = row['drl']
            if drl_score < 35:
                df.at[index, 'drltl'] = 0
            elif 35 <= drl_score < 50:
                df.at[index, 'drltl'] = 1
            elif 50 <= drl_score < 65:
                df.at[index, 'drltl'] = 2
            elif 65 <= drl_score < 80:
                df.at[index, 'drltl'] = 3
            elif 80 <= drl_score < 90:
                df.at[index, 'drltl'] = 4
            elif drl_score >= 90:
                df.at[index, 'drltl'] = 5

    df.to_excel(file, index=False)


In [ ]:
for file in files:
   if file in '10.diemrl.xlsx':
      df = pd.read_excel(file)
      tmp = ['id','ghichu','drl']
      df.drop(columns=tmp, inplace=True, errors='ignore')
      df.info()
      df.to_excel(file, index=False)
      break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54002 entries, 0 to 54001
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    54001 non-null  object 
 1   lopsh   54001 non-null  object 
 2   hocky   54001 non-null  float64
 3   namhoc  54001 non-null  float64
 4   drltl   3333 non-null   float64
dtypes: float64(3), object(2)
memory usage: 2.1+ MB


In [ ]:
for file in files:
   if file in 'diemrl.xlsx':
      df = pd.read_excel(file)
      tmp = ['id','drl','ghichu']
      df.drop(columns=tmp, inplace=True, errors='ignore')
      df.info()
      df.to_excel(file, index=False)
      break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111978 entries, 0 to 111977
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   mssv    111978 non-null  object
 1   lopsh   101450 non-null  object
 2   hocky   111978 non-null  int64 
 3   namhoc  111978 non-null  int64 
 4   drltl   111978 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.3+ MB


<h3> Bởi vì có 2 file điểm rèn luyện của 2 khoảng thời gian kết nhau nên em kết hợp lại </h3>

In [ ]:
diemrl = pd.read_excel('diemrl.xlsx')
diemrl2 = pd.read_excel('10.diemrl.xlsx')

if 'mssv' in diemrl.columns and 'mssv' in diemrl2.columns:
    combined_df = pd.concat([diemrl, diemrl2], ignore_index=True)

    combined_df.info()

    combined_df.to_excel('combined_diemrl.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165980 entries, 0 to 165979
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   mssv    165979 non-null  object 
 1   lopsh   155451 non-null  object 
 2   hocky   165979 non-null  float64
 3   namhoc  165979 non-null  float64
 4   drltl   115311 non-null  float64
dtypes: float64(3), object(2)
memory usage: 6.3+ MB


### Xử lý điểm trung bình

In [ ]:
for file in files:
   if file in 'sinhvien_dtb_hocky.xlsx':
      df = pd.read_excel(file)
      df.info()
      df.to_excel(file, index=False)
      break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84952 entries, 0 to 84951
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    84952 non-null  object 
 1   hocky   84952 non-null  int64  
 2   namhoc  84952 non-null  int64  
 3   dtbhk   84952 non-null  float64
 4   sotchk  84952 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 3.2+ MB


### Tiến hành join các file lại với nhau và xuất dưới dạng csv vì dạng xlsx không display được

In [ ]:
sinhvien_df = pd.read_excel('01.sinhvien.xlsx')
diem_df1 = pd.read_excel('04.xeploaiav.xlsx')
diem_df2 = pd.read_excel('05.ThiSinh.xlsx')
diem_df3 = pd.read_excel('combined_diemrl.xlsx')
diem_df4 = pd.read_excel('sinhvien_dtb_hocky.xlsx')

merged_df = pd.merge(sinhvien_df, diem_df1, on='mssv', how='outer')
# Check 1
merged_df = pd.merge(merged_df, diem_df2, on='mssv', how='outer')
# Check mssv
merged_df = pd.merge(merged_df, diem_df3, on='mssv', how='outer')
merged_df = pd.merge(merged_df, diem_df4, on=['mssv', 'hocky', 'namhoc'], how='outer')

merged_df.to_csv('final_merged_data.csv', index=False)

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172971 entries, 0 to 172970
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   mssv          172970 non-null  object 
 1   gioitinh      123204 non-null  float64
 2   lopsh_x       123204 non-null  object 
 3   khoahoc       123204 non-null  float64
 4   chuyennganh2  123204 non-null  object 
 5   hedt_ CLC     123205 non-null  object 
 6   hedt_ CNTN    123205 non-null  object 
 7   hedt_ CQUI    123205 non-null  object 
 8   hedt_ CTTT    123205 non-null  object 
 9   hedt_ KSTN    123205 non-null  object 
 10  khoa_ CNPM    123205 non-null  object 
 11  khoa_ HTTT    123205 non-null  object 
 12  khoa_ KHMT    123205 non-null  object 
 13  khoa_ KTMT    123205 non-null  object 
 14  khoa_ KTTT    123205 non-null  object 
 15  khoa_ MMT&TT  123205 non-null  object 
 16  khuvuc        123205 non-null  float64
 17  xl_av         72371 non-null   float64
 18  xepl

### Tiến hành cleaning bước cuối loại các giá trị trùng lặp và ngoại lai

In [ ]:
test_df = pd.read_csv('final_merged_data.csv')
test_df = test_df.drop_duplicates()
test_df.drop('lopsh_y',inplace=True,axis=1)
test_df.rename(columns={'lopsh_x': 'lopsh'}, inplace=True)
test_df = test_df.sort_values(by=['mssv', 'namhoc', 'hocky'])
test_df['dtbhk_truoc'] = test_df.groupby('mssv')['dtbhk'].shift(1)
for i in range(len(test_df)):
    if test_df.iloc[i]['hocky'] == 1 and (i == 0 or test_df.iloc[i]['mssv'] != test_df.iloc[i-1]['mssv']):
        test_df.at[test_df.index[i], 'dtbhk_truoc'] = None

test_df.info()

cols_to_check = [col for col in test_df.columns if col != 'xl_av']

missing_cols = [col for col in cols_to_check if col not in test_df.columns]
if missing_cols:
    print("check", missing_cols)
else:
    cleaned_df = test_df.dropna(subset=cols_to_check)
    cleaned_df.info()

cleaned_df.to_csv('final_final_final_merged_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 172971 entries, 0 to 172970
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   mssv          172970 non-null  object 
 1   gioitinh      123204 non-null  float64
 2   lopsh         123204 non-null  object 
 3   khoahoc       123204 non-null  float64
 4   chuyennganh2  123204 non-null  object 
 5   hedt_ CLC     123205 non-null  object 
 6   hedt_ CNTN    123205 non-null  object 
 7   hedt_ CQUI    123205 non-null  object 
 8   hedt_ CTTT    123205 non-null  object 
 9   hedt_ KSTN    123205 non-null  object 
 10  khoa_ CNPM    123205 non-null  object 
 11  khoa_ HTTT    123205 non-null  object 
 12  khoa_ KHMT    123205 non-null  object 
 13  khoa_ KTMT    123205 non-null  object 
 14  khoa_ KTTT    123205 non-null  object 
 15  khoa_ MMT&TT  123205 non-null  object 
 16  khuvuc        123205 non-null  float64
 17  xl_av         72371 non-null   float64
 18  xeploai_t

PermissionError: [Errno 13] Permission denied: 'final_final_final_merged_data.csv'

In [6]:
df = pd.read_csv('final_final_final_merged_data.csv')
df.head(5)

,mssv,gioitinh,lopsh,khoahoc,chuyennganh2,hedt_ CLC,hedt_ CNTN,hedt_ CQUI,hedt_ CTTT,hedt_ KSTN,...,khoa_ MMT&TT,khuvuc,xl_av,xeploai_tt,hocky,namhoc,drltl,dtbhk,sotchk,dtbhk_truoc
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,False,2.0,5.0,4.0,2.0,2019.0,5.0,9.00,21.0,8.84
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,False,2.0,5.0,4.0,1.0,2020.0,5.0,9.11,15.0,9.00
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,False,2.0,5.0,4.0,2.0,2020.0,5.0,8.75,19.0,9.11
3,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,False,2.0,5.0,4.0,1.0,2021.0,5.0,8.54,20.0,8.75
4,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,False,2.0,5.0,4.0,2.0,2021.0,5.0,8.74,20.0,8.54


In [ ]:
df['namnhaphoc'] = df['lopsh'].str.extract(r'(\d{4})')
df.head()


,mssv,gioitinh,lopsh,khoahoc,chuyennganh2,hedt_ CLC,hedt_ CNTN,hedt_ CQUI,hedt_ CTTT,hedt_ KSTN,...,khuvuc,xl_av,xeploai_tt,hocky,namhoc,drltl,dtbhk,sotchk,dtbhk_truoc,namnhaphoc
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2019.0,5.0,9.00,21.0,8.84,2019
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,1.0,2020.0,5.0,9.11,15.0,9.00,2019
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2020.0,5.0,8.75,19.0,9.11,2019
3,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,1.0,2021.0,5.0,8.54,20.0,8.75,2019
4,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2021.0,5.0,8.74,20.0,8.54,2019


In [10]:
condition = (df['namnhaphoc'] == df['namhoc']) & (df['hocky'] == 1)
num_rows_to_delete = condition.sum()


In [8]:
df = df[~((df['namnhaphoc'] == df['namhoc']) & (df['hocky'] == 1))]
df.head()

,mssv,gioitinh,lopsh,khoahoc,chuyennganh2,hedt_ CLC,hedt_ CNTN,hedt_ CQUI,hedt_ CTTT,hedt_ KSTN,...,khuvuc,xl_av,xeploai_tt,hocky,namhoc,drltl,dtbhk,sotchk,dtbhk_truoc,namnhaphoc
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2019.0,5.0,9.00,21.0,8.84,2019
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,1.0,2020.0,5.0,9.11,15.0,9.00,2019
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2020.0,5.0,8.75,19.0,9.11,2019
3,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,1.0,2021.0,5.0,8.54,20.0,8.75,2019
4,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,0.0,TMĐT2019,14.0,D52480104,False,False,True,False,False,...,2.0,5.0,4.0,2.0,2021.0,5.0,8.74,20.0,8.54,2019


In [11]:
output_file = "final_cuoi_data.csv"
df.to_csv(output_file, index=False)

In [13]:
td = pd.read_csv('final_cuoi_data.csv')
td['namnhaphoc'].unique()

array([2019,    1, 2018, 2013, 2017, 2014, 2016, 2015], dtype=int64)

In [14]:
td['namnhaphoc'] = td['namnhaphoc'].astype(str)
valid_rows = td['namnhaphoc'].str.match(r'^2\d{3}$')
num_rows_deleted = (~valid_rows).sum()
print(f"Số dòng bị xóa: {num_rows_deleted}")
td = td[valid_rows]
output_file = "cuoi_cung_ne.csv"
td.to_csv(output_file, index=False)



Số dòng bị xóa: 21271


In [ ]:
df = pd.read_csv("cuoi_cung_ne.csv")

mssv_to_drop = [
    "CC8ED1C2XPvAibaEXe9x0CaUg4pZo9/HVzJrxtyW",
    "CC1BE9F4XPvAibaEXe+NqszRBbPDty51v94rQVP2",
    "B37A5C4FXPvAibaEXe92uUBt7zfwrYq2UKjzgtwQ"
]

df_filtered = df[~df['mssv'].isin(mssv_to_drop)]

df_filtered.to_csv("cuoi_cung_thiet.csv", index=False, encoding='utf-8-sig')


Số dòng trước khi lọc: 35849
Số dòng sau khi lọc: 35826
File đã lưu: cuoi_cung_thiet.csv
